In [1]:
try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

# Math Library (openmdao.math)

Certain functions are useful in a gradient-based optimization context, such as smooth activation functions or differentiable maximum/minimum functions.

Rather than provide a component that forces a user to structure their system in a certain way and add more components than necessary, the `openmdao.math` package is intended to provide a universal source for _composable_ functions that users can use within their own components.

## Derivative functions

The functions provided in `openmdao.math` have corresponing analytic derivative functions.

These functions are vectorized in such a way that their jacobian matrix is typically diagonal, where the diagonal elements of the jacobian are nonzero and all off-diagonal elements are zero.
Instead of returning the entire dense jacobian, these derivative functions generally return the diagonal elements only.
There are some exceptions to this, such as `norm`, since it can return a non-diagonal jacobian when the `axis` argument is specified.

Derivatives of function outputs with respect to their inputs are returned by functions named
`d_{func_name}`. In addition to the arguments of the function, functions which accept more than one argument
provide boolean arguments which determine whether the derivative with respect to each argument is returned.

For instance, `d_smooth_abs(x, mu=1.0E-5, dmu=False)` returns the jacobian matrix for the smooth absolute value function with respect `x` and `None`, since the user disabled the calculation of the derivatives with respect to `mu`.  This can be useful when some arguments to the function are constants in the particular use-case.

In [17]:
import numpy as np
from openmdao.math import d_smooth_abs, d_arctan2

x = np.linspace(0, 5, 5)
print(f'x = {x}\n')

d_sabs_dx, d_sabs_dmu = d_smooth_abs(x, mu=1.0E-5, dmu=False)

print(f'd_smoothabs_dx = {d_sabs_dx}\n')
print(f'd_smoothabs_dmu = {d_sabs_dmu}\n')

x = [0.   1.25 2.5  3.75 5.  ]

d_smoothabs_dx = [0. 1. 1. 1. 1.]

d_smoothabs_dmu = None




A function will return a derivative for each output and each input. For instance, say we have
a function named `foo(x, y, z)` that returns two values, `a` and `b`.

The corresponding `d_foo(x, y, z)` will return 6 elements. Think of them as being in row-major
order where there is one row for each output and one column for each argument. Then we have

## Using `jax`

The `jax` Python package provides functionality that duplicates the math capability of numpy, but also supports automatic differentiation and just-in-time (jit) compilation.

By setting the environment variable `OPENMDAO_MATH_USE_JAX=1` (or to some other "truthy" value), the functions in `openmdao.math` will be built using `jax` rather than `numpy`, and therefore support `jax` features such as automatic differentiation and jit compilation.

```{eval-rst}\n",
    .. autofunction:: openmdao.math.act_tanh
```


## Derivatives

We wrap a function using the `omf.wrap` function, for example:
